# Assembling the StrokeMRI dataset

### Import libraries

In [ ]:
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt
import glob
import sys
sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config

## Read data into pandas dataframe

How do we define which files should be used together?
options:
- all files in folder
- based on suffix (e.g. "n=895_06-Feb-2023_PVC2.tsv")
- check first columns to see whether it matches

For now, we will will use the first option, approved by Mathijs on 26th June 2023

In [ ]:
experiment_folder= 'StrokeMRI_correct'

In [ ]:
config = Config()
root_directory = config.get_directory('raw_data')
if os.path.isdir(os.path.join(root_directory, experiment_folder)):
    print("this folder exists, we will take tsv from here")
else: 
    print("this folder does not seem to exist, try typing again")

In [ ]:
root_directory = config.get_directory('raw_data')
tsv_path = os.path.join(root_directory, experiment_folder)

tsv_files = [os.path.join(tsv_path, file) for file in os.listdir(tsv_path) if file.endswith('.tsv')]
tsv_files

In [ ]:
#CoV files -> COV values NOT 
tsv_files_for_cov = []
basics = []
for file in os.listdir(tsv_path):
    if file.startswith('CoV') and file.endswith('.tsv'):
        tsv_files_for_cov.append(os.path.join(tsv_path, file))
    elif file.endswith('.tsv'):
        basics.append(os.path.join(tsv_path, file))
basics      

In [ ]:
# Read files in selected folder into dataframes
cov_dataframes = [pd.read_csv(file, sep='\t', header=[0]) for file in tsv_files_for_cov]
# make a sample
sample_cov_df = cov_dataframes[2] # example of COV file
cols_cov = sample_cov_df.columns
#look at sample
sample_cov_df

In [ ]:
# Read files in selected folder into dataframes
basic_dataframes = [pd.read_csv(file, sep='\t', header=[0]) for file in basics]
# make a sample
sample_basic_df = basic_dataframes[2] # example of COV file
cols_basic = sample_basic_df.columns
#look at sample
sample_basic_df

In [ ]:
# Read ALL files in selected folder into dataframes
dataframes = [pd.read_csv(file, sep='\t', header=[0]) for file in tsv_files]
# make a sample
sample_df = dataframes[2] # example of file
cols = sample_df.columns
#look at sample
#sample_df

## Below we can see some different columns should be in our final dataframe except sex and age, which we add from another frame.
## These lists have some overlap as some values are CoV values that are in are basic values

In [ ]:
list_all_basic_column = []
for frame in basic_dataframes:
    list_columns = frame.columns.to_list()
    for x in list_columns:
        if x not in list_all_basic_column:
            list_all_basic_column.append(x)
#list_all_basic_column

In [ ]:
list_all_cov_column = []
for frame in cov_dataframes:
    list_columns = frame.columns.to_list()
    for x in list_columns:
        if x not in list_all_cov_column:
            list_all_cov_column.append(x)
#list_all_cov_column

## Now we need to translate all cov columns

In [ ]:
# we transform one of the COV, we need to transform all of them
for frame in cov_dataframes:
    print(frame.columns)

In [ ]:
dict_0 = {
    'SUBJECT' : 'participant_id', #
    'session' : 'session', #,
    'LongitudinalTimePoint' : 'LongitudinalTimePoint', #
    'SubjectNList' : 'SubjectNList', #
    'Site' : 'Site', #
    'GM_vol' : 'GM_vol', #
    'WM_vol' : 'WM_vol' ,#
    'CSF_vol' : 'CSF_vol', #
    'GM_ICVRatio' : 'GM_ICVRatio', #
    'GMWM_ICVRatio' : 'GMWM_ICVRatio', #
    'WMH_vol' : 'WMH_vol',#
    'WMH_count' : 'WMH_count', # to here
    'DeepWM_B' : 'DeepWM_B_cov',
    'DeepWM_L' : 'DeepWM_L_cov',
    'DeepWM_R' : 'DeepWM_R_cov',
       }
dict_1 = {
    'SUBJECT' : 'participant_id', #
    'session' : 'session', #,
    'LongitudinalTimePoint' : 'LongitudinalTimePoint', #
    'SubjectNList' : 'SubjectNList', #
    'Site' : 'Site', #
    'GM_vol' : 'GM_vol', #
    'WM_vol' : 'WM_vol' ,#
    'CSF_vol' : 'CSF_vol', #
    'GM_ICVRatio' : 'GM_ICVRatio', #
    'GMWM_ICVRatio' : 'GMWM_ICVRatio', #
    'WMH_vol' : 'WMH_vol',#
    'WMH_count' : 'WMH_count', # to here
    'ACA_B' : 'ACA_B_cov',
    'ACA_L' : 'ACA_L_cov',
    'ACA_R' : 'ACA_R_cov',
    'MCA_B' : 'MCA_B_cov',
    'MCA_L' : 'MCA_L_cov',
    'MCA_R' : 'MCA_R_cov',
    'PCA_B' : 'PCA_B_cov',
    'PCA_L' : 'PCA_L_cov',
    'PCA_R' : 'PCA_R_cov',
       }

dict_2 = {
    'SUBJECT' : 'participant_id', #
    'session' : 'session', #,
    'LongitudinalTimePoint' : 'LongitudinalTimePoint', #
    'SubjectNList' : 'SubjectNList',  #
    'Site' : 'Site', #
    'GM_vol' : 'GM_vol', #
    'WM_vol' : 'WM_vol' ,#
    'CSF_vol' : 'CSF_vol', #
    'GM_ICVRatio' : 'GM_ICVRatio', #
    'GMWM_ICVRatio' : 'GMWM_ICVRatio', #
    'WMH_vol' : 'WMH_vol',#
    'WMH_count' : 'WMH_count', # to here
    'TotalGM_B' : 'TotalGM_B_cov',
    'TotalGM_L' : 'TotalGM_L_cov',
    'TotalGM_R' : 'TotalGM_R_cov',
       }
cov_dataframes[0].rename(columns=dict_0,
          inplace=True)
cov_dataframes[1].rename(columns=dict_1,
          inplace=True)
cov_dataframes[2].rename(columns=dict_2,
          inplace=True)
cov_dataframes[0]

In [ ]:
away_subject_dict = {
    'SUBJECT' : 'participant_id', #
       }
for frame in basic_dataframes:
    print(frame)
    frame.rename(columns=away_subject_dict,
          inplace=True)

In [ ]:
numbr = 0
for frame in cov_dataframes:
    filepath = '../open_work/internal_results/stitchy/mri_cov' 
    filename = os.path.join(filepath,str(numbr+1)) 
    if not os.path.exists(filepath):
    # if filder doesn't exist, create it
        os.makedirs(filepath)
    frame.to_csv((filename +'.tsv'), sep="\t")
    numbr +=1

In [ ]:
numbr = 0
for frame in basic_dataframes:
    filepath = '../open_work/internal_results/stitchy/mri_basics' 
    filename = os.path.join(filepath,str(numbr+1)) 
    if not os.path.exists(filepath):
    # if filder doesn't exist, create it
        os.makedirs(filepath)
    frame.to_csv((filename +'.tsv'), sep="\t")
    numbr +=1

In [ ]:
cov_tsv_folder_made = '../open_work/internal_results/stitchy/mri_cov'
cov_identical_columns = sep.check_identical_columns(cov_tsv_folder_made)
cov_identical_columns

In [ ]:
basics_tsv_folder_made = '../open_work/internal_results/stitchy/mri_basics'
basics_identical_columns = sep.check_identical_columns(basics_tsv_folder_made)
basics_identical_columns

In [ ]:
# now we need to make a super dataframes list
super_dataframes = basic_dataframes +cov_dataframes
len(super_dataframes)

In [ ]:
identical_columns = sep.check_identical_columns(basics_tsv_folder_made)

In [ ]:
stitched = sample_basic_df[identical_columns].copy()

In [ ]:
#stitched = sample_basic_df[identical_columns].copy()

In [ ]:
stitched

In [ ]:
n_identical = stitched.shape[1]

In [ ]:
for df in super_dataframes:
    for col in df.columns[n_identical:]:
        stitched[col] = df[col]

stitched.columns

In [ ]:
len(stitched.columns)

In [ ]:
# now we can get rid of double header
stitched = stitched[1:]
#stitched

In [ ]:
stitched['session'].unique()

In [ ]:
stitched['LongitudinalTimePoint'].unique()

## Uhoh, we have now two time points.... we are going to get rid of the second, and tell the scientists later to confirm this was the right choice

In [ ]:
 stitched =stitched[stitched['LongitudinalTimePoint'].str.contains("1")]

In [ ]:
stitched['LongitudinalTimePoint'].unique()

In [ ]:
stitched.columns

In [ ]:
sexage_path = os.path.join(root_directory, 'age_data', 'Age_Sex_StrokeMRI.csv')
sexage_df = pd.read_csv(sexage_path, index_col=0)
sexage_df['renumber'] = sexage_df.index
sexage_df['renumber'] = sexage_df['renumber'].apply(str)
sexage_df

In [ ]:
## Here we take the patient ID and align it with our other frame's index

In [ ]:
stitched = stitched.reset_index(drop=False)
stitched

In [ ]:
len(stitched.participant_id.unique())

In [ ]:
stitched.participant_id

In [ ]:
stitched['renumber'] = stitched['participant_id']
stitched

In [ ]:
sexage_df = sexage_df.reset_index(drop=True)
sexage_df

# we need to recode renumber in the sexage df to match the renumber in the other result dataframe

In [ ]:
sexage_df['renumber'] = "sub-" + sexage_df['renumber'] +"_1"# sub-59080_1
sexage_df

In [ ]:
result = stitched.merge(sexage_df, on="renumber")
result

In [ ]:
# just in case there are duplicates in there
result = result.loc[:,~result.columns.duplicated()].copy()

## conform to new standard

In [ ]:
standard_path = '../researcher_interface/sample_sep_values/showable_standard.csv'


In [ ]:
standard = pd.read_csv(standard_path)
set_standard = set(standard.columns.to_list())

In [ ]:
result.columns = result.columns.str.lower()
set_results= set(result.columns.to_list())

z = set_results.intersection(set_standard) 
#z

In [ ]:
for f in result.columns:
    print(f)

In [ ]:
# shift column 'Name' to first position
first_column = result.pop('participant_id')
  
# insert column using insert(position,column_name,
# first_column) function
result.insert(0, 'participant_id', first_column)
result['participant_id'] = result['participant_id']+'_ses-1_run-1'
result

In [ ]:
result['session_id'] = result['session']
second_column = result.pop('session_id')
result.insert(1, 'session_id', second_column)
result['site'] = "MRI"
result['run_id'] = result['longitudinaltimepoint']
#result

## Here we need to confirm with scientisys that the MRI are all "reshuffled/normed"  values in the ASL

In [ ]:
result['gm_icv_ratio'] = result['gm_icvratio']
result['gmwm_icv_ratio'] = result['gm_icvratio']
result['cbf_gm_pvc0'] = 'NaN' 
result['cbf_gm_pvc2']=  result['totalgm_b'] 
result['cbf_wm_pvc0']= 'NaN'
result['cbf_wm_pvc2']= result['deepwm_b']
result['cbf_aca_pvc0'] =  'NaN'
result['cbf_mca_pvc0']  = 'NaN'
result['cbf_pca_pvc0'] =  'NaN'
result['cbf_aca_pvc2']  = result['aca_b']
result['cbf_mca_pvc2']  = result['mca_b']
result['cbf_pca_pvc2']  = result['pca_b']

In [ ]:
# for - need to check with student and scientists!
result['cov_gm_pvc0'] =  'NaN' # correct does not exist
# result['cov_gm_pvc2']  = result[]
result['cov_wm_pvc0']  = 'NaN' # does not exist
# result['cov_wm_pvc2']  = result[]
result['cov_aca_pvc0'] =  'NaN'# does not exist
result['cov_mca_pvc0']  = 'NaN'# does not exist
result['cov_pca_pvc0']  = 'NaN'# does not exist
result['cov_aca_pvc2'] = result['aca_b_cov']
result['cov_mca_pvc2'] = result['mca_b_cov']
result['cov_pca_pvc2'] = result['pca_b_cov']



In [ ]:
for f in result.columns:
    print(f)


In [ ]:
# we added a bunch of columns 
len(result.columns)

In [ ]:
sex_mapping = {1:'F',0:'M',}
results = result.assign(sex = result.sex.map(sex_mapping))
results

In [ ]:
standard.columns

In [ ]:
# shift columns to  position
third_column = results.pop('run_id')
fourth_column = results.pop('age')
fifth_column = results.pop('sex')
sixth_column = results.pop('site')
seventh_column = results.pop('gm_vol')
eight_column = results.pop('wm_vol')
ninth_column = results.pop('csf_vol')
tenth_column = results.pop('gm_icv_ratio')
eleventh_column = results.pop('gmwm_icv_ratio')
twelvth_column = results.pop('wmh_vol')
thirteenth_column = results.pop('wmh_count')

#last_column = results.pop('index')

results.insert(2, 'run_id', third_column)
results.insert(3, 'age', fourth_column)
results.insert(4, 'sex', fifth_column)
results.insert(5, 'site', sixth_column)
results.insert(6, 'gm_vol', seventh_column)
results.insert(7, 'wm_vol', eight_column)
results.insert(8, 'csf_vol', ninth_column)
results.insert(9, 'gm_ivc_ratio',tenth_column)
results.insert(10, 'gmwm_ivc_ratio',eleventh_column)
results.insert(11, 'wmh_vol',twelvth_column)
results.insert(12, 'wmh_count',thirteenth_column)
#results.insert(82, 'index', last_column)

results

In [ ]:
results = results.drop(['index', 'subjectnlist','session','longitudinaltimepoint', 'tp','gmwm_icvratio','gm_icvratio'], axis=1) 

In [ ]:
# for column in results.columns:
#     print(column, type(results[column][9]))


In [ ]:

standard.columns

In [ ]:
results.columns

In [ ]:
head_columns = results.columns[:13]
middle_columns = results.columns[13:44]
tail_columns = results.columns[44:]

In [ ]:
#tail_columns

In [ ]:
f_results = pd.concat([results[head_columns],results[tail_columns],results[middle_columns]], axis=1)
f_results.columns

In [ ]:
#standard.columns

In [ ]:
for column in f_results.columns:
    print(column)

In [ ]:
for column in f_results.columns[6:]:
    #print(column)
    f_results[column] = pd.to_numeric(f_results[column], errors = 'coerce')


In [ ]:
#f_results

In [ ]:
sep.check_sex_dimorph_expectations(f_results)

In [ ]:
len(f_results.columns.to_list())

In [ ]:
sep.relate_columns_graphs_numeric(f_results, 'age')

In [ ]:
doubled_outliers =sep.find_outliers_by_list(f_results, f_results.columns.to_list()[6:], 2.5)

In [ ]:
doubled_outliers

## Save outliers for M

In [ ]:
filepath = '../open_work/internal_results' 
filename = os.path.join(filepath,'possible_outliers_from_MRI.csv') 
if not os.path.exists(filepath):
    # if filder doesn't exist, create it
    os.makedirs(filepath)
doubled_outliers.to_csv(filename)  

In [ ]:
filepath = '../open_work/internal_results' 
filename = os.path.join(filepath,'mri_stitched_conformed.csv') 
if not os.path.exists(filepath):
    # if filder doesn't exist, create it
    os.makedirs(filepath)
f_results.to_csv(filename)  